In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold
from catboost import CatBoostRegressor
from skopt import BayesSearchCV

start_date = '2010-01-01'
end_date = '2020-01-01'
dates = pd.date_range(start=start_date, end=end_date, freq='D')
unix_dates = np.array([int(pd.Timestamp(d).timestamp()) for d in dates])
freq = 'D'  # Monthly frequency
date_rng = pd.date_range(start=start_date, end=end_date, freq=freq)
prices = np.sin(unix_dates)
data = pd.DataFrame({'ds': date_rng, 'y': prices})

In [24]:
# Prepare data
data['ds_numeric'] = data['ds'].apply(lambda x: x.value)  # Convert dates to numeric

In [25]:
X = data[['ds_numeric']] / 1e11

In [26]:
X

,ds_numeric
0,12623040.0
1,12623904.0
2,12624768.0
3,12625632.0
4,12626496.0
...,...
3648,15774912.0
3649,15775776.0
3650,15776640.0
3651,15777504.0


In [27]:
#X = data[['ds_numeric']]  # Input features
y = data['y']  # Target variable

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X.values.astype(int), y.values, test_size = 0.2, shuffle = False)

In [28]:
# Define the parameter search space for CatBoost
param_space = {
    'depth': (3, 12),
    'learning_rate': (0.01, 0.3),
    'n_estimators': (50, 500)
}

In [29]:
# Create CatBoost model
catboost_model = CatBoostRegressor(loss_function='RMSE', verbose=False)

In [30]:
# Create KFold cross-validation
kf = KFold(n_splits=5, shuffle=False)

In [31]:
# Perform Bayesian optimization with KFold cross-validation
bayes_cv_tuner = BayesSearchCV(
    estimator=catboost_model,
    search_spaces=param_space,
    n_iter=50,
    scoring='neg_mean_squared_error',
    cv=kf,
    n_jobs=-1
)

In [32]:
# Fit the model
bayes_cv_tuner.fit(X_train, y_train)

AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations